# Data preprocessing

In [0]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
path =  'x1_train.txt'
x1 = pd.read_csv(path, header=None, sep='\s+')
print(x1.shape)
path = 'y1_train.txt'
x2 = pd.read_csv(path, header=None, sep='\s+')
print(x2.shape)
path ='z1_train.txt'
x3 = pd.read_csv(path, header=None, sep='\s+')
print(x3.shape)
path =   'x2_train.txt'
x4 = pd.read_csv(path, header=None, sep='\s+')
print(x4.shape)
path = 'y2_train.txt'
x5 = pd.read_csv(path, header=None, sep='\s+')
print(x5.shape)
path = 'z2_train.txt'
x6 = pd.read_csv(path, header=None, sep='\s+')
print(x6.shape)
path =  'x3_train.txt'
x7 = pd.read_csv(path, header=None, sep='\s+')

path =  'y3_train.txt'
x8 = pd.read_csv(path, header=None, sep='\s+')

path =  'z3_train.txt'
x9 = pd.read_csv(path, header=None, sep='\s+')



print(x1.loc[1,1])
######
#reshape x
x1=x1.values.tolist()
x1=np.array(x1).reshape((7352,128,1))

x2=x2.values.tolist()
x2=np.array(x2).reshape((7352,128,1))

x3=x3.values.tolist()
x3=np.array(x3).reshape((7352,128,1))
x4=x4.values.tolist()
x4=np.array(x4).reshape((7352,128,1))

x5=x5.values.tolist()
x5=np.array(x5).reshape((7352,128,1))

x6=x6.values.tolist()
x6=np.array(x6).reshape((7352,128,1))
x7=x7.values.tolist()
x7=np.array(x7).reshape((7352,128,1))

x8=x8.values.tolist()
x8=np.array(x8).reshape((7352,128,1))

x9=x9.values.tolist()
x9=np.array(x9).reshape((7352,128,1))
X_train=np.concatenate((x1,x2,x3,x4,x5,x6,x7,x8,x9),axis=2)
print(X_train.shape)
####



(7352, 128)
(7352, 128)
(7352, 128)
(7352, 128)
(7352, 128)
(7352, 128)
0.0045500770000000005
(7352, 128, 9)


In [0]:
def convert_Z(z):
    a=np.zeros((z.shape[0],30))
    for j in range(0,z.shape[0]):
      for i in range(30):
        if z[j,0]==i+1:
                a[j,i]=1
    return a  

In [0]:
path = 'subject_train.txt'
Z_train = pd.read_csv(path, header=None)
print(Z_train)
Z_train=convert_Z(np.array(Z_train))
print(np.sum(Z_train,axis=0))

Z_train=Z_train[:,np.sum(Z_train,axis=0)!=0]

       0
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
...   ..
7322  30
7323  30
7324  30
7325  30
7326  30
7327  30
7328  30
7329  30
7330  30
7331  30
7332  30
7333  30
7334  30
7335  30
7336  30
7337  30
7338  30
7339  30
7340  30
7341  30
7342  30
7343  30
7344  30
7345  30
7346  30
7347  30
7348  30
7349  30
7350  30
7351  30

[7352 rows x 1 columns]
[347.   0. 341.   0. 302. 325. 308. 281.   0.   0. 316.   0.   0. 323.
 328. 366. 368.   0. 360.   0. 408. 321. 372.   0. 409. 392. 376. 382.
 344. 383.]


In [0]:
path =  'y_train.txt'
Y_train = pd.read_csv(path, header=None)

print(Y_train.describe())
Y_train=Y_train.values.tolist()
Y_train=np.array(Y_train)
print(Y_train.shape)

                 0
count  7352.000000
mean      3.643362
std       1.744802
min       1.000000
25%       2.000000
50%       4.000000
75%       5.000000
max       6.000000
(7352, 1)


In [0]:
l=[]
for i in range(Y_train.shape[0]):
  if Y_train[i] ==4 or Y_train[i] ==5:
    l.append(i)
             

In [0]:
X=X_train[np.array(l),:,:]
Y=Y_train[np.array(l),]
Z=Z_train[np.array(l),]
Z.shape

(2660, 21)

In [0]:
def convert_to_label(y):
    a=np.zeros((y.shape[0],1))
    for j in range(0,y.shape[0]):
        if y[j,0]==4:
                a[j,0]=1
    return a  

In [0]:
label=convert_to_label(Y)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test,z_train, z_test = train_test_split(X, label,Z ,test_size=0.2, random_state=42)





In [0]:
np.sum(z_test,axis=0)

array([14., 25., 22., 21., 28., 18., 15., 26., 21., 33., 24., 37., 34.,
       22., 29., 27., 31., 30., 34., 21., 20.])

# CNN

In [0]:

n_classes = 1
n_channels = 9

seq_len = 128          # Number   time sequence
learning_rate = 0.0001
epochs = 1200
keep_prob_=0.5
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
 # (batch, 128, 9) --> (batch, 64, 18)
conv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=16, strides=1, 
                             padding='same', activation = tf.nn.relu)
max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 64, 18) --> (batch, 32, 36)
conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=36, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 32, 36) --> (batch, 16, 72)
conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=72, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 16, 72) --> (batch, 8, 144)
conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=144, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')

# Flatten and add dropout
flat = tf.reshape(max_pool_4, (-1, 8*144))
flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
# Predictions
logits = tf.layers.dense(flat, n_classes)
prediction=tf.nn.sigmoid(logits)
# Cost function and optimizer
cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels_,logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

 # Calculate the correct predictions
correct_prediction = tf.to_float(tf.greater(prediction, 0.5))
accuracy = tf.reduce_mean(tf.to_float(tf.equal(labels_, correct_prediction)))
ACC=[]
for i in range(20):
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #iteration = 1
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        
      loss, _ =sess.run([cost, optimizer], feed_dict={inputs_ : X_train, labels_: y_train})
            
            
    
            
            # Print at each 5 iters
      if (e % 50 == 0):
        acc=sess.run(accuracy, feed_dict={inputs_ : X_test, labels_: y_test})
        print("Epoch: {}/{}".format(e, epochs),
             "Train loss: {:6f}".format(loss),
             "TEST acc: {:.6f}".format(acc))
    ACC.append(acc)            
                

Epoch: 0/1200 Train loss: 0.693210 TEST acc: 0.473684
Epoch: 50/1200 Train loss: 0.605877 TEST acc: 0.744361
Epoch: 100/1200 Train loss: 0.373594 TEST acc: 0.855263
Epoch: 150/1200 Train loss: 0.298192 TEST acc: 0.857143
Epoch: 200/1200 Train loss: 0.282210 TEST acc: 0.866541
Epoch: 250/1200 Train loss: 0.275248 TEST acc: 0.874060
Epoch: 300/1200 Train loss: 0.260892 TEST acc: 0.874060
Epoch: 350/1200 Train loss: 0.259311 TEST acc: 0.862782
Epoch: 400/1200 Train loss: 0.249737 TEST acc: 0.883459
Epoch: 450/1200 Train loss: 0.240640 TEST acc: 0.877820
Epoch: 500/1200 Train loss: 0.233910 TEST acc: 0.881579
Epoch: 550/1200 Train loss: 0.224688 TEST acc: 0.875940
Epoch: 600/1200 Train loss: 0.209468 TEST acc: 0.877820
Epoch: 650/1200 Train loss: 0.200167 TEST acc: 0.875940
Epoch: 700/1200 Train loss: 0.187563 TEST acc: 0.894737
Epoch: 750/1200 Train loss: 0.166269 TEST acc: 0.902256
Epoch: 800/1200 Train loss: 0.150553 TEST acc: 0.909774
Epoch: 850/1200 Train loss: 0.132961 TEST acc: 0.93

KeyboardInterrupt: ignored

In [0]:
ACC

# CNN COMBINED WITH GLMM

In [0]:
n_classes = 1
n_channels = 9

seq_len = 128          # Number   time sequence
learning_rate = 0.0001
epochs = 150
keep_prob_=0.5
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
Y_ = tf.placeholder(tf.float32, [None, 1], name = 'y')
 # (batch, 128, 9) --> (batch, 64, 18)
conv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=16, strides=1, 
                             padding='same', activation = tf.nn.relu)
max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 64, 18) --> (batch, 32, 36)
conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=36, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 32, 36) --> (batch, 16, 72)
conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=72, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 16, 72) --> (batch, 8, 144)
conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=144, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')
#print(max_pool_3.shape[-2])
# Flatten and add dropout
flat = tf.reshape(max_pool_4, (-1, 8*144))
flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    

logits = tf.layers.dense(flat, n_classes)

# Cost function and optimizer
cost=cost=tf.losses.mean_squared_error(Y_,logits)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


## Taingning data

In [0]:
X=np.ones((y_train.shape[0],1))
Y=y_train.reshape(y_train.shape[0],)
Z=z_train
X.shape,Y.shape,Z.shape

((2128, 1), (2128,), (2128, 21))

## Testing data

In [0]:
Ytest=y_test.reshape(y_test.shape[0],)
Xtest=np.ones((y_test.shape[0],1)).astype(float)
Ztest=z_test
#X_test #ppg signal
print(Ytest.shape,Xtest.shape,Ztest.shape)

(532,) (532, 1) (532, 21)


In [0]:
def lamda(eita):
    s=1/(1+np.exp(-eita))
    lamda=(1/(2*eita))*(s-0.5)
    return lamda


def Estep(beta_old,sigma_old,Y,X,Z,epsilon):#Input old parameters and X,Y,Z
    N=Y.shape[0]
    #L=X.shape[1]
    K=Z.shape[1]
    Sigma=(1/sigma_old)*(np.ones(K))
    
    for i in range(N):
        Sigma=Sigma+2*lamda(np.sqrt(epsilon[i]))*Z[i,]#(np.outer(Z[i,],Z[i,]).diagonal())
    
    Sigma1=1/(Sigma)
    mu=np.zeros(K)
    for i in range(N):
        mu=mu+Z[i,]-2*Y[i]*Z[i,]+4*lamda(np.sqrt(epsilon[i]))*(np.dot(X[i,],beta_old))*Z[i,]
    p=-0.5*Sigma1*mu
    list1=[p,Sigma1]  #OUTPUT： POSTERIOR mean and covariance matrix
    return list1


In [0]:
#Initialization
beta_old=np.random.random(X.shape[1])
sigma_old=1
gamma_old=1
b_old=np.append(beta_old,gamma_old)
N=X.shape[0]
epsilon=np.zeros(N)

for i in range(N):
  epsilon[i]=sigma_old*np.dot(Z[i,],Z[i,])+(np.dot(X[i,],beta_old))**2

p,Sigma=Estep(beta_old,sigma_old,Y,X,Z,epsilon)#initialize E step
epsilon_new=epsilon#updatew epsilon
for i in range(N):
    epsilon_new[i]=np.dot(Sigma,Z[i,])+(np.dot(p,Z[i,]))**2+2*(np.dot(X[i,],beta_old))*(np.dot(Z[i,],p))+(np.dot(X[i,],beta_old))**2
g=-1000000000#lower bound

In [0]:
def y(beta_old,gamma_old,Y,X,Z,epsilon):
    N=X.shape[0]
    y_nn=np.zeros(N)
    for i in range(N):
        y_nn[i]=0.5/(lamda(np.sqrt(epsilon[i]))*gamma_old)*(Y[i]-0.5-2*lamda(np.sqrt(epsilon[i]))*(np.dot(X[i,],beta_old)+np.dot(Z[i,],p)))
    return y_nn   
        
def bMstep(beta_old,sigma_old,Y,X,Z,epsilon,p,Sigma):
    epsilon_new=epsilon
    N=X.shape[0]
    L=X.shape[1]
    K=Z.shape[1]  
    S=np.zeros((L,L))
    M=np.zeros(L)
    for i in range(N):
        S=S+2*lamda(np.sqrt(epsilon_new[i]))*np.outer(X[i,],X[i,])
        M=M+Y[i]*X[i,]-2*lamda(np.sqrt(epsilon_new[i]))*(np.dot(Z[i,],p))*X[i,]-0.5*X[i,]
    beta_new=np.matmul(np.linalg.inv(S),M)
    sigma_new=(np.dot(p,p)+np.sum(Sigma))/K
    list3=[beta_new,sigma_new]
    return list3        
        

In [0]:
def prediction(x,beta,p,Sigma,z):#  Ep_i
    mu=np.dot(z,p)
    sigma=np.dot(Sigma,z)# posterior mean and sigma of random effects
    def w(u):#pi
        b=1/(1+np.exp(-np.dot(x,beta)-u))
        return b
    W=np.zeros(2000)
    for i in range(2000):
        u=np.random.normal(mu, sigma, 1)# Monte carlo method
        W[i]=w(u)
    return np.mean(W)
def accuracy(X,Y,Z,beta_old,p,Sigma):
  N=X.shape[0]
  acc=0
  Y_P=np.zeros(N)
  for i in range(N):
    if prediction(X[i,],beta_old,p,Sigma,Z[i,])>0.5:
      Y_P[i]=1
    if Y_P[i]==Y[i]:
      acc=acc+1
      
  return acc/N  

In [0]:
ACC2=[]
for i in range(20):
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    
    N=Y.shape[0]
    L1=X.shape[1]
    K=Z.shape[1] 
    # Loop over epochs
    for e in range(epochs):
          
    
   
    #update cost and logits
        epsilon=epsilon_new
        Y_new=y(beta_old,gamma_old,Y,X,Z,epsilon)#  X , beta_old not x_com. b_old
        #cost=tf.squre(Y_-logits)
        
        for u in range(100):
          sess.run(optimizer, feed_dict={inputs_ :  X_train, Y_: Y_new.reshape(Y_new.shape[0],1)})#feed Y_:Y_neW
       
        logit,costs=sess.run([logits,cost], feed_dict={inputs_ :  X_train, Y_: Y_new.reshape(Y_new.shape[0],1)})
        print(costs)
        
        b_old=np.append(beta_old,gamma_old)
        X_com=np.concatenate((X,logit),axis=1)
        b_old,sigma_old=bMstep(b_old,sigma_old,Y,X_com,Z,epsilon,p,Sigma)
        beta_old=b_old[0:L1]
        gamma_old=b_old[-1]
        g_old=g
        g=-0.5*K*np.log(sigma_old)+0.5*np.sum(np.log(Sigma))#lower bound
        for i in range(N):
            g=g+Y[i]*(np.dot(Z[i,],p)+np.dot(X_com[i,],b_old))-lamda(np.sqrt(epsilon[i]))*(np.dot(Sigma,Z[i,])+(np.dot(p,Z[i,]))**2+2*(np.dot(X_com[i,],b_old))*(np.dot(Z[i,],p))+(np.dot(X_com[i,],b_old))**2-epsilon[i])-0.5*(np.dot(Z[i,],p))-0.5*(np.dot(X_com[i,],b_old))-0.5*np.sqrt(epsilon[i])+np.log(1/(1+np.exp(-np.sqrt(epsilon[i]))))
    #if g。g_
        if abs(g-g_old)<0.05:
          break
        #TEST ACCURACY
        #Xtest =featuretest+1,Ytest=labeltest
        if (e%50==49):
          logitstest=sess.run(logits, feed_dict={inputs_ :  X_test})
          X_comtest=np.concatenate((Xtest,logitstest),axis=1)
          
          acc=accuracy(X_com,Y,Z,b_old,p,Sigma)
          print(acc,sigma_old)
        #E step
          p,Sigma=Estep(b_old,sigma_old,Y,X_com,Z,epsilon)#initialize
         #updateepsilon
          epsilon_new=epsilon
        for i in range(N):
            epsilon_new[i]=np.dot(Sigma,Z[i,])+(np.dot(p,Z[i,]))**2+2*(np.dot(X_com[i,],b_old))*(np.dot(Z[i,],p))+(np.dot(X_com[i,],b_old))**2
        
        print(g)
    ACC2.append(acc)        

In [0]:
 ACC2